### Лекция 7 - ООП в Python
- особенности инкапсуляции, наследования и полиморфизма в Python
- динамическое создание классов
- метаклассы
- MRO
- ```__slots__```
- пользовательские исключения
- менеджеры контекста
- глубокое копирование
- декораторы через классы
- класс enum
- модуль attrs

In [17]:
# staticmethod, classmethod, abstractmethod
# private members - name mangling: _CLASS__private

Что хотим сделать (аналог на C#)
```
interface IParser
{
	void Load();
	string Parse();	
}

class XMLParser : IParser
{
	void Load() { ... }
	string Parse() { ... }
}

class JSONParser : IParser
{
	void Load() { ... }
	string Parse() { ... }
}
```

#### Динамическое создание классов и метаклассы

In [16]:
Person = type('Person', (object,), 
              {'name': 'vasya', 
               'age': 21, 
               'calc': lambda self: self.age + 1})
p = Person()
print(p.name)
print(p.calc())

# class Person(object):
#     def __init__(self):
#         self.name = 'vasya'
#         self.age = 21
#
#     def calc(self):
#         self.age += 1
#

dir(p)

vasya
22


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'age',
 'calc',
 'name']

In [34]:
# Пример метакласса
# http://eli.thegreenplace.net/2011/08/14/python-metaclasses-by-example

class PointMeta(type):
    def __new__(meta, name, bases, dct):
        print('-----------------------------------')
        print('Allocating memory for class', name)
        print(meta)
        print(bases)
        print(dct)
        return super(PointMeta, meta).__new__(meta, name, bases, dct)
    
    def __init__(cls, name, bases, dct):
        print('\n-----------------------------------')
        print('Initializing class', name)
        print(cls)
        print(bases)
        print(dct)
        super(PointMeta, cls).__init__(name, bases, dct)
        
    def __call__(cls, *args, **kwds):
        print('__call__ of ', str(cls))
        print('__call__ *args=', str(args))
        print()
        return type.__call__(cls, *args, **kwds)

In [35]:
class Point(metaclass=PointMeta):
    
    # в Python 2 пишется здесь:
    #__metaclass__ = PointMeta

    def __init__(self, x, y):
        print('Point p({}, {})'.format(x, y))

        
pt = Point(42, 73)

-----------------------------------
Allocating memory for class Point
<class '__main__.PointMeta'>
()
{'__qualname__': 'Point', '__init__': <function Point.__init__ at 0x02A06078>, '__module__': '__main__'}

-----------------------------------
Initializing class Point
<class '__main__.Point'>
()
{'__qualname__': 'Point', '__init__': <function Point.__init__ at 0x02A06078>, '__module__': '__main__'}
__call__ of  <class '__main__.Point'>
__call__ *args= (42, 73)

Point p(42, 73)


#### ```__slots__```

The special attribute ```__slots__``` allows you to explicitly state in your code which instance attributes you expect your object instances to have, with the expected results:

- faster attribute access.
- potential space savings in memory.

And the biggest caveat for multiple inheritance - multiple "parent classes with nonempty slots" cannot be combined. (Solution? Factor out all but one (or just all) parents' abstraction which they respectively and you collectively will inherit from - giving the abstraction(s) empty slots.)

Requirements:

To have attributes named in ```__slots__``` to actually be stored in slots instead of a ```__dict__```, a class must inherit from object.
To prevent the creation of a ```__dict__```, you must inherit from object and all classes in the inheritance must declare ```__slots__``` and none of them can have a ```'__dict__'``` entry - and they cannot use multiple inheritance.

http://stackoverflow.com/questions/472000/usage-of-slots

By default, instances of both old and new-style classes have a dictionary for attribute storage. This wastes space for objects having very few instance variables. The space consumption can become acute when creating large numbers of instances.

The default can be overridden by defining ```__slots__``` in a new-style class definition. The ```__slots__``` declaration takes a sequence of instance variables and reserves just enough space in each instance to hold a value for each variable. Space is saved because ```__dict__``` is not created for each instance.

SQLAlchemy attributes a lot of memory savings with ```__slots__```.

In [6]:
class SmallObject(object):
    
    __slots__ = ['width', 'height', 'path']
    
    def __init__(self):
        self.width = 10
        self.height = 10
        self.path = 'default'
    
    @property
    def size(self):
        return self.width * self.height
        

obj = SmallObject()
print(obj.path)
print(obj.size)

default
100


#### Пример пользовательского исключения

In [8]:
class FileTooBigError(Exception):

    def __init__(self, filesize):
        super(FileTooBigError, self).__init__(
            'Filesize is too big: {} bytes'.format(filesize))
        self.filesize = filesize

    # если __str__() должен возвращать просто сообщение message, 
    # то метод можно не определять 
    # (по умолчанию, строковое представление Exception - и так message)
    # def __str__(self):
    #    return self.message


import os

fsize = os.path.getsize(r'lec01 - Intro.ipynb')

try:
    if fsize > 4096:
        raise FileTooBigError(fsize)
except FileTooBigError as err:
    print(err)

Filesize is too big: 34227 bytes


#### Менеджеры контекста

In [13]:
# ПЕРВЫЙ СПОСОБ:

class PingContext(object):

    def __enter__(self):
        self.log = open(r'data\log.txt', 'wt')
        print('Log file is ready')
        return self

    def write(self, text):
        self.log.write(text)
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_value != None:
            print('Ping failed! Wrong IP format: %s' % exc_value)
            # подчистить ресурсы, которыми владеет контекст
        self.log.close()
        print('Log file is closed')    
        return True
        #return False          # если False, то заново бросается Exception
                               # (и надо будет перехватывать во внешнем коде)


IPs = ['127.0.0.1', '0.0.0.0', '1.1.1.1', '10.0.0.1', 'f.2.3.4', '192.168.0.102']

# регулярка для простейшей проверки валидности IP-адреса
import re
regex = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')

with PingContext() as log:
    # проходим по всем записям из строк IP-адресов
    for IP in IPs:
        if not re.match(regex, IP):
            raise ValueError(IP)
        # пинг IP-адреса ...
        # и запись в лог
        log.write(IP + '\n')

Log file is ready
Ping failed! Wrong IP format: f.2.3.4
Log file is closed


In [3]:
# ВТОРОЙ СПОСОБ:

from contextlib import contextmanager

@contextmanager
def ping_log():
    # __enter__
    print('Log file is ready')
    log = open(r'data\log.txt', 'wt')

    # actions
    try:
        yield log

    # __exit__    
    except ValueError as err:
        print('Ping failed! Wrong IP format: %s' % err)
    finally:
        log.close()
        print('Log file is closed')


regex = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')

IPs = ['127.0.0.1', '0.0.0.0', '1.1.1.1', '10.0.0.1', 'f.2.3.4', '192.168.0.102']

with ping_log() as log:
    for IP in IPs:
        if not re.match(regex, IP):
            raise ValueError(IP)
        log.write(IP + '\n')

Log file is ready
Ping failed! Wrong IP format: f.2.3.4
Log file is closed


#### Глубокое копирование

In [1]:
# демо глубокого копирования стандартных типов
import copy

l1 = [1, "Hello", [3,4,5], (2, [7,8,9], 'x'), 12]

l2 = copy.deepcopy(l1)
l2[1] = "World"

print(l1)
print(l2)

[1, 'Hello', [3, 4, 5], (2, [7, 8, 9], 'x'), 12]
[1, 'World', [3, 4, 5], (2, [7, 8, 9], 'x'), 12]


In [9]:
# демо глубокого копирования объектов классов
from fractions import Fraction

f1 = Fraction(3, 5)
f2 = copy.deepcopy(f1)  # дефолтная реализация;
                        # ее можно переопределить в классе Fraction: 
                        # метод __deepcopy__(self, f)
print(f1)
print(f2)

3/5
3/5


In [10]:
# мы тоже написали ранее класс Fraction,
# но есть стандартный класс дробей в модуле fractions:
dir(f1)

['__abs__',
 '__abstractmethods__',
 '__add__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__complex__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_add',
 '_denominator',
 '_div',
 '_mul',
 '_numerator',
 '_operator_fallbacks',
 '_richcmp',
 '_sub',
 'conjugate',
 'denominator',
 'from_decimal',
 'from_float',

#### Декоратор через класс

In [64]:
import functools

class Prettifier(object):
    def __init__(self, char='*'):
        self.char = char
        
    def __call__(self, func):
        def decorated(*args, **kwargs):
            print(self.char * len(args[0]))
            func(*args, **kwargs)
            print(self.char * len(args[0]))
        return decorated

@Prettifier('=')
def function(text):
    print(text)
    

function('decoratin\'!')

decoratin'!
